In [7]:
pip install pandas transformers


In [8]:
from google.colab import drive
drive.mount('/content/drive')

Drive already mounted at /content/drive; to attempt to forcibly remount, call drive.mount("/content/drive", force_remount=True).


In [9]:
import csv
import re
import string
import pandas as pd
from transformers import AutoTokenizer, AutoModelForSequenceClassification
from transformers import pipeline

df_ytb = pd.read_csv("/content/drive/MyDrive/CSV FILES/ytb_Little Miss Sunshine.csv")

# Remove column name 'Unnamed: 0'
dfc = df_ytb.drop(['Unnamed: 0','username'], axis=1)
# dfc = df_ytb.drop(index=11)
dfc = dfc.reset_index(drop=True)

#list_of_sentences = dfc['comment'].tolist()
#dfc
def clean_html(text):
    clean=re.compile('<.*?>')
    cleantext=re.sub(clean,'',text)
    return cleantext

# first round of cleaning
def clean_text1(text):
    text=text.lower()
    text=re.sub('','',text)
    text=re.sub('[%s]'%re.escape(string.punctuation),'',text)
    text=re.sub('\w*\d\w*','',text)
    return text

# second round of cleaning
def clean_text2(text):
    text=re.sub('[''"",,,]','',text)
    text=re.sub('\n','',text)
    return text

cleaned_html=lambda x:clean_html(x)
cleaned1=lambda x:clean_text1(x)
cleaned2=lambda x:clean_text2(x)

dfc['review'] = dfc['comment']
dfc['review']=pd.DataFrame(dfc.review.apply(cleaned_html))
dfc['review']=pd.DataFrame(dfc.review.apply(cleaned1))
dfc['review']=pd.DataFrame(dfc.review.apply(cleaned2))
print(dfc.shape)
dfc.head(10)

first10comments = dfc.review.values

sentences = first10comments
list_of_sentences = dfc['comment'].tolist()


(100, 2)


In [10]:
tokenizer = AutoTokenizer.from_pretrained("aiknowyou/it-emotion-analyzer")
model = AutoModelForSequenceClassification.from_pretrained("aiknowyou/it-emotion-analyzer")

emotion_analysis = pipeline("sentiment-analysis", model=model, tokenizer=tokenizer)
emotion_analysis(list_of_sentences)
results = emotion_analysis(list_of_sentences)
results

[{'label': '1', 'score': 0.672942042350769},
 {'label': '1', 'score': 0.7087196111679077},
 {'label': '5', 'score': 0.5226147174835205},
 {'label': '2', 'score': 0.5101103782653809},
 {'label': '2', 'score': 0.5856441855430603},
 {'label': '1', 'score': 0.4865027964115143},
 {'label': '1', 'score': 0.9245933294296265},
 {'label': '3', 'score': 0.4454953968524933},
 {'label': '2', 'score': 0.9319642186164856},
 {'label': '1', 'score': 0.34156593680381775},
 {'label': '2', 'score': 0.8696420192718506},
 {'label': '2', 'score': 0.8788131475448608},
 {'label': '1', 'score': 0.7771520614624023},
 {'label': '1', 'score': 0.5575226545333862},
 {'label': '1', 'score': 0.3733847737312317},
 {'label': '2', 'score': 0.735384464263916},
 {'label': '2', 'score': 0.9376624822616577},
 {'label': '2', 'score': 0.8610480427742004},
 {'label': '2', 'score': 0.7293003797531128},
 {'label': '2', 'score': 0.9448357224464417},
 {'label': '2', 'score': 0.9591200947761536},
 {'label': '2', 'score': 0.66036880

In [11]:
emotion_mapping = {
    0: 'sadness',
    1: 'joy',
    2: 'love',
    3: 'anger',
    4: 'fear',
    5: 'surprise'
}
# Map numerical labels to emotions in the results
dfc['Emotion'] = [emotion_mapping.get(result['label'], result['label']) for result in results]
dfc['Score'] = [result['score'] for result in results]

final =dfc[['comment', 'Emotion', 'Score']]
final


,comment,Emotion,Score
0,"what a nice timing pop <a href=""https://www.yo...",1,0.672942
1,I have this film in my dvd collection watching...,1,0.708720
2,Freaking Richard...,5,0.522615
3,My favorite scene and one of the best lines I’...,2,0.510110
4,"When I was in my 20s, I beat the absolute piss...",2,0.585644
...,...,...,...
95,The emo kid looks like the one in Horton hears...,2,0.649889
96,"This movie was so good...the script, the actin...",2,0.545460
97,Olive was the only one who was related to just...,2,0.846430
98,I wanna see this movie so bad... it looks awesome,2,0.776880


In [12]:
# Create a new column 'mapped_emotion' using emotion_mapping
emotion_mapping = {
    0: 'sadness',
    1: 'joy',
    2: 'love',
    3: 'anger',
    4: 'fear',
    5: 'surprise'
}
final['Emotion'] = pd.to_numeric(final['Emotion'], errors='coerce')

final['Mapped_Emotion'] = final['Emotion'].map(emotion_mapping)

# Display the updated 'final' DataFrame with the new column
print(final[['Emotion', 'Mapped_Emotion']])

    Emotion Mapped_Emotion
0         1            joy
1         1            joy
2         5       surprise
3         2           love
4         2           love
..      ...            ...
95        2           love
96        2           love
97        2           love
98        2           love
99        1            joy

[100 rows x 2 columns]


<ipython-input-12-a9bc13b09b5c>:10: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  final['Emotion'] = pd.to_numeric(final['Emotion'], errors='coerce')


In [13]:
# Add sentiment analysis results to the DataFrame
# Save the DataFrame with sentiment results to a new CSV file
output_csv_path = "/content/drive/MyDrive/ytb_Little Miss Sunshine_with_sentiments.csv"
final.to_csv(output_csv_path, index=False)

# Display the first few rows of the DataFrame with sentiment results
final

,comment,Emotion,Score,Mapped_Emotion
0,"what a nice timing pop <a href=""https://www.yo...",1,0.672942,joy
1,I have this film in my dvd collection watching...,1,0.708720,joy
2,Freaking Richard...,5,0.522615,surprise
3,My favorite scene and one of the best lines I’...,2,0.510110,love
4,"When I was in my 20s, I beat the absolute piss...",2,0.585644,love
...,...,...,...,...
95,The emo kid looks like the one in Horton hears...,2,0.649889,love
96,"This movie was so good...the script, the actin...",2,0.545460,love
97,Olive was the only one who was related to just...,2,0.846430,love
98,I wanna see this movie so bad... it looks awesome,2,0.776880,love
